### Импортим либы

In [2]:
import random
import math
from typing import Callable
import tabulate

### Делаем два генератора случайных величин с равномерным и экспоненциальным распределением

Вызывать так:
``` python
obj = even_int_gen(3, 5)
next(obj)
# 3, или 4, или даже 5...
# Жеееесть...
```

In [3]:
def even_int_gen(a: int, b: int) -> int:

    a -= 0.5 
    b += 0.5
    while True:
        yield round(a + random.random() * (b - a))


def exp_int_gen(l: float) -> int:
    while True:
        yield (-1) * math.log(random.random()) / l

### Напишем функцию, делающую аналитический расчет вероятности для системы СМО Эрланга

In [4]:
def erlang_model(l: float, u: float, stands_count: int) -> float:
    alpha = l / u
    n = stands_count
    up = alpha ** n / math.factorial(n)
    down = 1
    for i in range(1, n + 1):
        down += alpha ** i / math.factorial(i)
    return 1 - up / down

### Тестим

In [10]:
l = 4.0
u = 3.0

table = []
for chan in range(1, 8):
    table.append([chan, erlang_model(l, u, chan)])

print("λ =", l)
print("μ =", u)

headers = ["# канала обслуживания", "% контроля"]
print(tabulate.tabulate(table, headers=headers, tablefmt="grid", floatfmt=".5f"))

λ = 4.0
μ = 3.0
+-------------------------+--------------+
|   # канала обслуживания |   % контроля |
+=========================+==============+
|                       1 |      0.42857 |
+-------------------------+--------------+
|                       2 |      0.72414 |
+-------------------------+--------------+
|                       3 |      0.89078 |
+-------------------------+--------------+
|                       4 |      0.96487 |
+-------------------------+--------------+
|                       5 |      0.99072 |
+-------------------------+--------------+
|                       6 |      0.99794 |
+-------------------------+--------------+
|                       7 |      0.99961 |
+-------------------------+--------------+


### Напишем функцию имитационного моделирования

На вход идет:
1. **items_count** - количество деталей для обработки
2. **stands_count** - количество каналов (стендов) для обработки
3. **release_gen** - генератор значений времени входного канала
4. **check_gen** - генератор значений времени обработки на стенде

Засчет того, что мы даем функции на вход генераторы, то можно дать как генератор равномерно распределенных чисел, так и генератор экспоненциально распределенных чисел.

Это позволит легко, без переписывания функции в дальнейшем, отладить ее работу, сравнивая значения с СМО Эрланга, которая использует экспоненциальное распределение, а нам в дальнейшем понадобится равномерное.

In [6]:

def imit_model(items_count: int, stands_count: int,
               release_gen: Callable[[int, int], int],
               check_gen: Callable[[int, int], int]) -> float:

    stands = [0 for _ in range(stands_count)]
    not_checked = 0

    for i in range(items_count):
        rel_time = next(release_gen)

        for j in range(len(stands)):
            stands[j] = max(0, stands[j] - rel_time)

        found_free = False
        for j in range(len(stands)):
            if stands[j] == 0:
                stands[j] = next(check_gen)
                found_free = True
                break

        if not found_free:
            not_checked += 1

    return (items_count - not_checked) / items_count

### Тестим

In [7]:
l = 4
u = 3
release_gen = exp_int_gen(l)
check_gen = exp_int_gen(u)

need = 0.98
items_count = 10**5
stands_count = 1

table = []
while True:
    result = imit_model(items_count, stands_count, release_gen, check_gen)
    erl_result = erlang_model(l, u, stands_count)
    table.append([stands_count, result, erl_result])
    stands_count += 1
    if result >= need:
        break

print("λ =", l)
print("μ =", u)
print("Количество изделий на входе:", 50)
print("Контроль не менее", str(need * 100), "% изделий")

headers = ["n_k", "имитация", "Эрланг"]
print(tabulate.tabulate(table, headers=headers, tablefmt="fancy_grid", floatfmt=".5f"))

λ = 4
μ = 3
Количество изделий на входе: 100000
Контроль не менее 98.0 % изделий
╒═══════╤════════════╤══════════╕
│   n_k │   имитация │   Эрланг │
╞═══════╪════════════╪══════════╡
│     1 │    0.42864 │  0.42857 │
├───────┼────────────┼──────────┤
│     2 │    0.72578 │  0.72414 │
├───────┼────────────┼──────────┤
│     3 │    0.89088 │  0.89078 │
├───────┼────────────┼──────────┤
│     4 │    0.96567 │  0.96487 │
├───────┼────────────┼──────────┤
│     5 │    0.99019 │  0.99072 │
╘═══════╧════════════╧══════════╛


Как видно, имитация показывает верный результат относительно аналитического решения СМО Эрланга

### Проверим систему с равномерным распределением

In [11]:
a, b = 8, 18
c, d = 18, 28
release_gen = even_int_gen(a, b)
check_gen = even_int_gen(c, d)

need = 0.8
items_count = 10**5
stands_count = 1

table = []
while True:
    result = imit_model(items_count, stands_count, release_gen, check_gen)
    table.append([stands_count, result])
    stands_count += 1
    if result >= need:
        break

print("a, b =", [a, b])
print("c, d =", [c, d])
print("Количество изделий на входе:", items_count)
print("Контроль не менее", str(need * 100), "% изделий")

headers = ["номер канала", "%"]
print(tabulate.tabulate(table, headers=headers, tablefmt="fancy_grid", floatfmt=".5f"))

a, b = [8, 18]
c, d = [18, 28]
Количество изделий на входе: 100000
Контроль не менее 80.0 % изделий
╒════════════════╤═════════╕
│   номер канала │       % │
╞════════════════╪═════════╡
│              1 │ 0.44218 │
├────────────────┼─────────┤
│              2 │ 0.84090 │
╘════════════════╧═════════╛
